## Imports & Setup

In [3]:
import pandas as pd
import numpy as np
import os
import glob
import matplotlib.pyplot as plt
import seaborn as sns
import gc

# Config
RAW_DIR = os.path.join(os.getcwd(), "data", "raw_full")
PROCESSED_DIR = os.path.join(os.getcwd(), "data", "processed")
os.makedirs(PROCESSED_DIR, exist_ok=True)

# Threshold: Drop columns with > 50% missing values
MISSING_THRESHOLD = 0.5 

# Helper Function: Reduce Memory Usage 
def optimize_dtypes(df):
    """
    Converts float64 -> float32 and int64 -> int32 to save memory.
    """
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type == 'float64':
            df[col] = df[col].astype('float32')
        elif col_type == 'int64':
            df[col] = df[col].astype('int32')
            
    return df

print(f"📂 Reading from: {RAW_DIR}")
print(f"🎯 Saving to: {PROCESSED_DIR}")

📂 Reading from: /workspaces/MAP_Hackathon_2026_G4/src/data/raw_full
🎯 Saving to: /workspaces/MAP_Hackathon_2026_G4/src/data/processed


## Load All Data

In [4]:
# 1. Get all file paths
all_files = glob.glob(os.path.join(RAW_DIR, "*.xpt")) + glob.glob(os.path.join(RAW_DIR, "*.XPT"))
demo_path = next((f for f in all_files if "DEMO" in os.path.basename(f).upper()), None)

if not demo_path:
    raise FileNotFoundError("CRITICAL: DEMO file not found!")

print(f"⚓ Anchor File: {os.path.basename(demo_path)}")
df_master = pd.read_sas(demo_path)

# Optimize Anchor immediately
df_master['SEQN'] = df_master['SEQN'].astype('int32')
df_master = optimize_dtypes(df_master)
base_seqns = set(df_master['SEQN'])

print(f"   Initial Shape: {df_master.shape}")
print(f"   Memory (Optimized): {df_master.memory_usage().sum() / 1024**2:.2f} MB")

⚓ Anchor File: DEMO_J.xpt
   Initial Shape: (9254, 46)
   Memory (Optimized): 1.62 MB


## Memory-Optimized Iterative Merge

In [5]:
count = 0
dropped_vars_count = 0

print("🚀 Starting iterative merge with compression...")

for f in all_files:
    if f == demo_path: continue
    
    # Garbage collection before heavy lifting
    gc.collect()
    
    try:
        # A. Read file
        df_temp = pd.read_sas(f)
        
        # B. Check SEQN
        if 'SEQN' not in df_temp.columns:
            del df_temp
            continue
            
        # Optimize SEQN immediately for matching
        df_temp['SEQN'] = df_temp['SEQN'].astype('int32')
        
        # C. Row Filter (Keep only valid participants)
        df_temp = df_temp[df_temp['SEQN'].isin(base_seqns)]
        
        if df_temp.empty:
            del df_temp
            continue
            
        # D. Column Filter (Drop empty columns)
        cols_to_keep = ['SEQN']
        # Calculate missingness
        # Note: We compute this on the FILTERED rows, which is more accurate
        missing = df_temp.isnull().mean()
        valid_cols = missing[missing <= MISSING_THRESHOLD].index.tolist()
        
        # Ensure we don't duplicate columns already in master (except SEQN)
        new_cols = [c for c in valid_cols if c not in df_master.columns or c == 'SEQN']
        
        if len(new_cols) <= 1: # Only SEQN remains
            del df_temp
            continue
            
        # Keep only valid, new columns
        df_temp = df_temp[new_cols]
        
        # E. COMPRESS DATA (The Fix)
        df_temp = optimize_dtypes(df_temp)
        
        # F. Merge
        df_master = pd.merge(df_master, df_temp, on='SEQN', how='left')
        
        # G. Cleanup
        del df_temp
        count += 1
        
        if count % 10 == 0:
            mem_usage = df_master.memory_usage().sum() / 1024**2
            print(f"   ... Merged {count} files. Shape: {df_master.shape}, Mem: {mem_usage:.2f} MB")
            
    except Exception as e:
        print(f"⚠️ Error {os.path.basename(f)}: {e}")
        # Ensure cleanup even on error
        if 'df_temp' in locals(): del df_temp
        gc.collect()

🚀 Starting iterative merge with compression...
   ... Merged 10 files. Shape: (17529, 757), Mem: 50.69 MB


: 

In [ ]:
import pandas as pd
import numpy as np
import os
import glob
import gc
import shutil

# === Configuration ===
RAW_DIR = os.path.join(os.getcwd(), "data", "raw_full")
TEMP_DIR = os.path.join(os.getcwd(), "data", "temp_parts") # Intermediate storage
PROCESSED_DIR = os.path.join(os.getcwd(), "data", "processed")

# Create/Reset directories
if os.path.exists(TEMP_DIR):
    shutil.rmtree(TEMP_DIR)
os.makedirs(TEMP_DIR, exist_ok=True)
os.makedirs(PROCESSED_DIR, exist_ok=True)

MISSING_THRESHOLD = 0.5 

# === Helper: Reduce Memory Usage ===
def optimize_dtypes(df):
    for col in df.columns:
        if df[col].dtype == 'float64':
            df[col] = df[col].astype('float32')
        elif df[col].dtype == 'int64':
            df[col] = df[col].astype('int32')
    return df

# ==========================================
# PHASE 1: PRE-PROCESS TO DISK
# ==========================================
print("🚀 PHASE 1: Pre-processing files to disk...")

# 1. Setup Anchor (DEMO)
all_files = glob.glob(os.path.join(RAW_DIR, "*.xpt")) + glob.glob(os.path.join(RAW_DIR, "*.XPT"))
demo_path = next((f for f in all_files if "DEMO" in os.path.basename(f).upper()), None)

print(f"⚓ Loading Anchor: {os.path.basename(demo_path)}")
df_demo = pd.read_sas(demo_path)
df_demo['SEQN'] = df_demo['SEQN'].astype('int32')
base_seqns = set(df_demo['SEQN'])  # The master list of valid IDs

# Save DEMO immediately and clear memory
df_demo = optimize_dtypes(df_demo)
df_demo.to_pickle(os.path.join(TEMP_DIR, "000_DEMO.pkl"))
print(f"   -> Saved Anchor. Rows: {len(df_demo)}")
del df_demo
gc.collect()

# 2. Process all other files
saved_count = 0
for i, f in enumerate(all_files):
    if f == demo_path: continue
    fname = os.path.basename(f)
    
    try:
        # Load ONE file
        df_temp = pd.read_sas(f)
        
        # Validation
        if 'SEQN' not in df_temp.columns:
            del df_temp
            continue
            
        # Filter Rows (Keep only valid participants)
        df_temp['SEQN'] = df_temp['SEQN'].astype('int32')
        df_temp = df_temp[df_temp['SEQN'].isin(base_seqns)]
        
        if df_temp.empty:
            del df_temp
            continue

        # Filter Columns (Drop empty ones)
        # We calculate missingness on the valid rows only
        missing = df_temp.isnull().mean()
        valid_cols = missing[missing <= MISSING_THRESHOLD].index.tolist()
        
        # Must keep SEQN
        if 'SEQN' not in valid_cols: valid_cols.append('SEQN')
        
        # If the file has no useful data, skip it
        if len(valid_cols) <= 1: 
            del df_temp
            continue
            
        # Keep only valid cols & Compress
        df_temp = df_temp[valid_cols]
        df_temp = optimize_dtypes(df_temp)
        
        # SAVE to disk and delete from RAM
        save_name = f"{i:03d}_{fname.replace('.xpt', '').replace('.XPT', '')}.pkl"
        df_temp.to_pickle(os.path.join(TEMP_DIR, save_name))
        
        saved_count += 1
        if saved_count % 10 == 0:
            print(f"   Processed {saved_count} files...")
            
        del df_temp
        gc.collect()
        
    except Exception as e:
        print(f"   ⚠️ Skipped {fname}: {e}")
        gc.collect()

print(f"✅ Phase 1 Complete. {saved_count} clean files saved to disk.")

# ==========================================
# PHASE 2: MERGE FROM DISK
# ==========================================
print("\n🚀 PHASE 2: Merging cleaned files...")

# Load the anchor back
df_master = pd.read_pickle(os.path.join(TEMP_DIR, "000_DEMO.pkl"))

# Get all temp files (excluding DEMO)
temp_files = sorted(glob.glob(os.path.join(TEMP_DIR, "*.pkl")))
temp_files = [f for f in temp_files if "000_DEMO" not in f]

for i, f in enumerate(temp_files):
    try:
        # Load small clean chunk
        df_part = pd.read_pickle(f)
        
        # Remove duplicate columns if any (keep Master's version)
        cols_to_use = df_part.columns.difference(df_master.columns).tolist()
        cols_to_use.append('SEQN')
        
        # Merge
        df_master = pd.merge(df_master, df_part[cols_to_use], on='SEQN', how='left')
        
        # Cleanup
        del df_part
        if i % 10 == 0:
            print(f"   Merged {i+1}/{len(temp_files)}... Current Shape: {df_master.shape}")
            gc.collect()
            
    except Exception as e:
        print(f"   Merge Error on {os.path.basename(f)}: {e}")

# ==========================================
# PHASE 3: FINAL OUTPUT
# ==========================================
print("\n" + "="*40)
print(f"🎉 GRAND TOTAL: {df_master.shape}")

# Save final
final_pkl = os.path.join(PROCESSED_DIR, "nhanes_final.pkl")
df_master.to_pickle(final_pkl)
print(f"💾 Saved Pickle: {final_pkl}")

# Clean up temp files
shutil.rmtree(TEMP_DIR)
print("🧹 Temp files cleaned up.")

## Final Save

In [ ]:
print("\n" + "="*40)
print(f"🎉 Merge Complete!")
print(f"📊 Final Shape: {df_master.shape}")
print(f"🗑️ Total Variables Dropped (Pre-filter): {dropped_vars_count}")

# Save as CSV
csv_path = os.path.join(PROCESSED_DIR, "nhanes_2017_2018_filtered.csv")
df_master.to_csv(csv_path, index=False)
print(f"💾 Saved CSV to: {csv_path}")

# Save as Pickle (Recommended for Python workflows - reads/writes much faster)
pkl_path = os.path.join(PROCESSED_DIR, "nhanes_2017_2018_filtered.pkl")
df_master.to_pickle(pkl_path)
print(f"💾 Saved Pickle to: {pkl_path}")

## Calculate Missing Rates (Memory Safe)

In [ ]:
import pandas as pd
import numpy as np
import os
import glob
import matplotlib.pyplot as plt
import gc  # Garbage Collection

# === Configuration ===
# Path to your raw XPT files
RAW_DIR = os.path.join(os.getcwd(), "data", "raw_full")
MISSING_THRESHOLD = 0.5  # 50% threshold

print(f"📂 Scanning files in: {RAW_DIR}")
all_files = glob.glob(os.path.join(RAW_DIR, "*.xpt")) + glob.glob(os.path.join(RAW_DIR, "*.XPT"))
print(f"🔍 Found {len(all_files)} files. Calculating missing rates iteratively...")

var_missing_rates = {}

for i, f in enumerate(all_files):
    try:
        # Read one file at a time
        df_temp = pd.read_sas(f)
        
        # Calculate missing rate for each variable in this file
        # This returns a Series like: {'SEQN': 0.0, 'LBXGLU': 0.1, ...}
        rates = df_temp.isnull().mean()
        
        # Store in our master dictionary
        # Note: If a variable appears in multiple files (like SEQN), this will overwrite with the latest.
        # Since variables are mostly unique to specific tables, this is fine for a general overview.
        var_missing_rates.update(rates.to_dict())
        
        # Free memory immediately
        del df_temp
        gc.collect()
        
        # Progress indicator
        if (i + 1) % 20 == 0:
            print(f"   Processed {i + 1}/{len(all_files)} files...")
            
    except Exception as e:
        print(f"⚠️ Error reading {os.path.basename(f)}: {e}")

# Convert dictionary to Series for your plotting code
missing_series = pd.Series(var_missing_rates)

print(f"\n✅ Analysis Complete!")
print(f"   Total Variables Analyzed: {len(missing_series)}")
print(f"   Variables to Keep (< {MISSING_THRESHOLD*100}% missing): {len(missing_series[missing_series <= MISSING_THRESHOLD])}")
print(f"   Variables to Drop (> {MISSING_THRESHOLD*100}% missing): {len(missing_series[missing_series > MISSING_THRESHOLD])}")

## Visualization

In [ ]:
# what kept vs dropped
plt.figure(figsize=(10, 5))
plt.hist(missing_series * 100, bins=20, color='skyblue', edgecolor='black')
plt.axvline(MISSING_THRESHOLD * 100, color='red', linestyle='--', label=f'Cutoff ({MISSING_THRESHOLD*100}%)')
plt.title('Distribution of Missing Data % Across All Variables')
plt.xlabel('% Missing')
plt.ylabel('Number of Variables')
plt.legend()
plt.show()

## Filter by Missing Values

In [ ]:
print("🧹 Starting Filtration...")

# 1. Calculate missing rates
missing_series = df_master.isnull().mean()

# 2. Identify keep vs drop
keep_cols = missing_series[missing_series <= MISSING_THRESHOLD].index
drop_cols = missing_series[missing_series > MISSING_THRESHOLD].index

print(f"   Total Variables: {len(missing_series)}")
print(f"   Keep (<= {MISSING_THRESHOLD*100}% missing): {len(keep_cols)}")
print(f"   Drop (> {MISSING_THRESHOLD*100}% missing): {len(drop_cols)}")

# 3. Create filtered DataFrame
df_clean = df_master[keep_cols].copy()

# 4. Save
save_path = os.path.join(PROCESSED_DIR, "nhanes_2017_2018_filtered.csv")
# Use pickle for faster IO in Python, or CSV for compatibility
df_clean.to_csv(save_path, index=False)
df_clean.to_pickle(os.path.join(PROCESSED_DIR, "nhanes_2017_2018_filtered.pkl"))

print(f"\n🎉 Saved clean data to: {save_path}")
print(f"📊 Final Clean Shape: {df_clean.shape}")

## Visualize Missing Data

In [ ]:
# Let's see what we kept vs dropped
plt.figure(figsize=(10, 5))
plt.hist(missing_series * 100, bins=20, color='skyblue', edgecolor='black')
plt.axvline(MISSING_THRESHOLD * 100, color='red', linestyle='--', label=f'Cutoff ({MISSING_THRESHOLD*100}%)')
plt.title('Distribution of Missing Data % Across All Variables')
plt.xlabel('% Missing')
plt.ylabel('Number of Variables')
plt.legend()
plt.show()